In [2]:
using ReactiveMP, Rocket, GraphPPL, Random, LinearAlgebra, Plots, Flux, Zygote, ForwardDiff, DataFrames, DelimitedFiles

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1423
┌ Warning: Package ReactiveMP does not have Flux in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Flux as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Flux into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.


In [18]:
function f(x, y, z)
    return x + 2 * y + z
end

@model function multiple_interfaces(N)
    
    μ ~ NormalMeanVariance(0, 1)
    z = randomvar(N)
    x = randomvar(N)
    y = randomvar(N)
    t = randomvar(N)
    out = datavar(Float64, N)

    for i in 1:N
        z[i] ~ NormalMeanVariance(μ, 1)
        x[i] ~ NormalMeanVariance(μ, 1)
        y[i] ~ NormalMeanVariance(μ, 1)
        t[i] ~ f(x[i], y[i], z[i])
        # t[i] ~ ((x[i] + (2 * y[i])) + z[i])
        out[i] ~ NormalMeanVariance(t[i], 1)
    end

    return μ, z, x, y, out
end
constraints = @constraints begin
    q(z, x, y, μ, t) = q(μ)q(z)q(x)q(y)q(t)
end;

meta = @meta begin
   f(x, y, z) -> CVIApproximation(100, 100, ADAM(), 100, 20)
end

In [19]:
N = 1000
data = rand(NormalMeanVariance(), N);

res = inference(
    model = Model(multiple_interfaces, N),
    data = (out = data,),
    iterations = 2,
    free_energy = false,
    returnvars = (μ = KeepEach(),),
    constraints = constraints,
    meta = meta,
    options = (limit_stack_depth = 1000,),
    #initmessages = (z=NormalMeanVariance(), x=NormalMeanVariance(), y=NormalMeanVariance(),),
    initmarginals = (z=NormalMeanVariance(), x=NormalMeanVariance(), y=NormalMeanVariance(), μ=NormalMeanVariance()),
)

LoadError: Cluster intersection error in the expression `t[1] ~ f(x[1], y[1], z[1])`.
Based on factorisation constraints the resulting local constraint q(t[1], x[1], y[1], z[1]) = q(t[1], x[1])q(t[1], y[1])q(t[1], z[1])q(t[1], x[1], y[1], z[1]) has cluster intersections, which is disallowed by default.
Technical info: clusters = Tuple[(1, 2), (1, 3), (1, 4), (1, 2, 3, 4)]
Constraints:
  marginals form:
  messages form:
  factorisation:
    q(z, x, y, μ) = q(μ)q(z)q(x)q(y)
Options:
  warn = true
